In [1]:
!pip install huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 189.7 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [2]:
!nvidia-smi

Thu Jun  5 08:25:51 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.90.07              Driver Version: 550.90.07      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A10G                    On  |   00000000:00:1E.0 Off |                    0 |
|  0%   27C    P8             23W /  300W |       1MiB /  23028MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [3]:
!pip install --upgrade pip

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 4.8 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 24.2
    Uninstalling pip-24.2:
      Successfully uninstalled pip-24.2


In [9]:
!pip install -r requirements.txt

INFO: pip is looking at multiple versions of sentence-transformers to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of sentence-transformers to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 173.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 225.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 213.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 212.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 225.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 233.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 392.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 229.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## Snapshot Download of Granite Model

Download full repo to prepare for quantization.

In [12]:
# Downloading granite model
from huggingface_hub import snapshot_download

model_id = "ibm-granite/granite-3.2-8b-instruct"
local_path = "./granite-3.2-8b-instruct"

print(f"⬇️  Downloading model '{model_id}' to '{local_path}' ...")
local_path = snapshot_download(repo_id=model_id, local_dir=local_path, local_dir_use_symlinks=False)
print(f"✅ Model downloaded to: {local_path}")


/opt/app-root/lib64/python3.11/site-packages/huggingface_hub/file_download.py:980: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(


⬇️  Downloading model 'ibm-granite/granite-3.2-8b-instruct' to './granite-3.2-8b-instruct' ...


Fetching 15 files: 100%|██████████| 15/15 [03:13<00:00, 12.91s/it]

✅ Model downloaded to: /opt/app-root/src/SEAK_AI/Chatbot/granite-3.2-8b-instruct


## Testing Model

In [7]:
!nvidia-smi

Mon Apr 21 02:14:45 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.124.06             Driver Version: 570.124.06     CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       On  |   00000000:00:1C.0 Off |                    0 |
| N/A   41C    P0             28W /   70W |    1075MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [8]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

model_name = "granite-3.2-8b-instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype="auto"
)

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer
)

# Convert your chat-style messages into a single prompt string
prompt = "Are you better than chatgpt"

response = pipe(prompt, max_new_tokens=128, do_sample=True)
print(response[0]['generated_text'])


2025-04-21 02:14:49.029294: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-21 02:14:49.045673: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745201689.065932    3830 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745201689.072223    3830 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1745201689.087737    3830 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

Are you better than chatgpt?

ChatGPT is a model from OpenAI, while I am a model from IBM, specifically designed for business use cases. Each model has its own strengths and is fine-tuned for different types of tasks. I specialize in handling enterprise-level data and transactions securely and efficiently, providing accurate responses in a business context.

To provide a more direct comparison, ChatGPT is known for its versatility in generating human-like text across a wide range of topics, making it suitable for conversational AI applications. Meanwhile, my primary focus is on delivering precise, domain-specific responses


In [9]:
!nvidia-smi # Can see from here that model has been loaded into GPU memory

Mon Apr 21 02:17:01 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.124.06             Driver Version: 570.124.06     CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       On  |   00000000:00:1C.0 Off |                    0 |
| N/A   45C    P0             44W /   70W |   13401MiB /  15360MiB |     72%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


## Saving model in Minio

In [13]:
import os
import boto3
import botocore

# Retrieve AWS credentials and S3 connection details from environment variables
aws_access_key_id = os.environ.get('AWS_ACCESS_KEY_ID')
aws_secret_access_key = os.environ.get('AWS_SECRET_ACCESS_KEY')
endpoint_url = os.environ.get('AWS_S3_ENDPOINT')
region_name = os.environ.get('AWS_DEFAULT_REGION')
bucket_name = os.environ.get('AWS_S3_BUCKET')

print(bucket_name)

demo-chatbot


In [14]:
import os
import boto3
import botocore



local_model_dir = "granite-3.2-8b-instruct"  # Path to your local model directory
s3_prefix = "models/granite-model"   # Desired path in S3 bucket

# === Initialize session and resource ===
session = boto3.session.Session(
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key
)

s3_resource = session.resource(
    's3',
    config=botocore.client.Config(signature_version='s3v4'),
    endpoint_url=endpoint_url,
    region_name=region_name
)

bucket = s3_resource.Bucket(bucket_name)


def should_upload(file_path):
    """
    Determine if a file should be uploaded (skip .git and hidden files).
    """
    # Skip hidden files and folders (like .git, .DS_Store)
    parts = file_path.split(os.sep)
    for part in parts:
        if part.startswith('.'):
            return False
    return True


def upload_file_to_s3(local_file_path, s3_key):
    """
    Uploads a single file to S3.
    """
    print(f"Uploading {local_file_path} to s3://{bucket_name}/{s3_key}")
    bucket.upload_file(local_file_path, s3_key)
    print(f"✅ Uploaded to s3://{bucket_name}/{s3_key}")


def upload_directory_to_s3(local_directory, s3_prefix):
    """
    Recursively uploads a directory to S3, skipping unwanted files.
    """
    found_files = False

    for root, dirs, files in os.walk(local_directory):
        # Modify dirs in-place to skip hidden folders
        dirs[:] = [d for d in dirs if not d.startswith('.')]

        for filename in files:
            if filename.startswith('.'):
                continue  # Skip hidden files

            local_path = os.path.join(root, filename)
            if not should_upload(local_path):
                continue  # Skip unwanted files

            relative_path = os.path.relpath(local_path, local_directory)
            s3_key = os.path.join(s3_prefix, relative_path).replace("\\", "/")

            upload_file_to_s3(local_path, s3_key)
            found_files = True

    if not found_files:
        print("🚨 No valid files found to upload. Check your directory and filters!")
    else:
        print("✅ Upload completed.")


def list_objects(prefix):
    """
    Lists all objects in the S3 bucket with a given prefix.
    """
    print(f"\n📦 Objects in S3 under '{prefix}':")
    for obj in bucket.objects.filter(Prefix=prefix):
        print(obj.key)


# === Run Upload ===
upload_directory_to_s3(local_model_dir, s3_prefix)

# === List uploaded objects ===
list_objects(s3_prefix)


Uploading granite-3.2-8b-instruct/vocab.json to s3://demo-chatbot/models/granite-model/vocab.json
✅ Uploaded to s3://demo-chatbot/models/granite-model/vocab.json
Uploading granite-3.2-8b-instruct/merges.txt to s3://demo-chatbot/models/granite-model/merges.txt
✅ Uploaded to s3://demo-chatbot/models/granite-model/merges.txt
Uploading granite-3.2-8b-instruct/model-00003-of-00004.safetensors to s3://demo-chatbot/models/granite-model/model-00003-of-00004.safetensors
✅ Uploaded to s3://demo-chatbot/models/granite-model/model-00003-of-00004.safetensors
Uploading granite-3.2-8b-instruct/model.safetensors.index.json to s3://demo-chatbot/models/granite-model/model.safetensors.index.json
✅ Uploaded to s3://demo-chatbot/models/granite-model/model.safetensors.index.json
Uploading granite-3.2-8b-instruct/model-00002-of-00004.safetensors to s3://demo-chatbot/models/granite-model/model-00002-of-00004.safetensors
✅ Uploaded to s3://demo-chatbot/models/granite-model/model-00002-of-00004.safetensors
Uplo

In [1]:
from transformers import AutoTokenizer

# Download and cache the tokenizer to a local folder
AutoTokenizer.from_pretrained("ibm-granite/granite-3.2-8b-instruct", cache_dir="./tokenizer")


/opt/app-root/lib64/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


GPT2TokenizerFast(name_or_path='ibm-granite/granite-3.2-8b-instruct', vocab_size=49152, model_max_length=9223372036854775807, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<|end_of_text|>', 'eos_token': '<|end_of_text|>', 'unk_token': '<|end_of_text|>', 'pad_token': '<|end_of_text|>', 'additional_special_tokens': ['<|start_of_role|>', '<|end_of_role|>', '<|tool_call|>']}, clean_up_tokenization_spaces=True, added_tokens_decoder={
	0: AddedToken("<|end_of_text|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<fim_prefix>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("<fim_middle>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("<fim_suffix>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	4: AddedToken("<fim_pad>", rstrip=False, lstrip=False, single_word=False, n